In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests
import warnings
warnings.filterwarnings('ignore')

# ===============================
# 1. CSV 파일 로드
# ===============================
csv_file = 'crime_rate_long.csv'

try:
    merged_df = pd.read_csv(csv_file, encoding='utf-8-sig')
    print(f"✓ CSV 로드 완료: {len(merged_df)}개 레코드")
    print(f"  연도 범위: {merged_df['year'].min()} - {merged_df['year'].max()}")
    print(f"  구 개수: {merged_df['region'].nunique()}개")
except FileNotFoundError:
    print(f"오류: {csv_file} 파일을 찾을 수 없습니다.")
    print("먼저 CSV 생성 코드를 실행해주세요.")
    exit()

# ===============================
# 2. 호버 텍스트 생성
# ===============================
merged_df['text'] = merged_df.apply(
    lambda x: f"<b>{x['region']}</b><br>범죄율: {x['crime_rate']:.1f}<br>범죄 건수: {x['total']:,}건<br>인구: {x['population']:,.0f}명",
    axis=1
)

# ===============================
# 3. 서울시 GeoJSON 다운로드
# ===============================
geojson_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

try:
    response = requests.get(geojson_url)
    seoul_geo = response.json()
    print("✓ GeoJSON 로드 완료")

    # GeoJSON에서 구별 중심 좌표 추출
    centroids = {}
    for feature in seoul_geo['features']:
        name = feature['properties']['name']
        coords = feature['geometry']['coordinates'][0]

        if feature['geometry']['type'] == 'MultiPolygon':
            coords = feature['geometry']['coordinates'][0][0]

        lats = [coord[1] for coord in coords]
        lons = [coord[0] for coord in coords]
        centroids[name] = {
            'lat': sum(lats) / len(lats),
            'lon': sum(lons) / len(lons)
        }

    merged_df['lat'] = merged_df['region'].map(lambda x: centroids.get(x, {}).get('lat'))
    merged_df['lon'] = merged_df['region'].map(lambda x: centroids.get(x, {}).get('lon'))

except Exception as e:
    print(f"⚠ 오류: GeoJSON 로드 실패 - {e}")
    seoul_geo = None

# ===============================
# 4. Plotly Choropleth 지도 생성
# ===============================
if seoul_geo:
    # 색상 범위를 분위수 기준으로 조정
    all_rates = merged_df['crime_rate'].values
    q25 = np.percentile(all_rates, 25)
    q75 = np.percentile(all_rates, 75)

    fig = go.Figure()

    # 각 연도별 프레임 생성
    years = sorted(merged_df['year'].unique())

    for year in years:
        year_data = merged_df[merged_df['year'] == year].copy()

        # Choropleth 레이어
        fig.add_trace(go.Choroplethmapbox(
            geojson=seoul_geo,
            locations=year_data['region'],
            z=year_data['crime_rate'],
            featureidkey="properties.name",
            colorscale='YlOrRd',
            zmin=q25,
            zmax=q75,
            marker_opacity=0.7,
            marker_line_width=1,
            marker_line_color='white',
            text=year_data['text'],
            hovertemplate='%{text}<extra></extra>',
            colorbar=dict(
                title="범죄율<br>(10만명당)",
                thickness=15,
                len=0.7,
                x=1.02
            ),
            name=str(year),
            visible=(year == years[4])  # 2018년을 기본으로 표시
        ))

        # 각 구에 텍스트 라벨 추가 (지역명만 표시)
        fig.add_trace(go.Scattermapbox(
            lat=year_data['lat'],
            lon=year_data['lon'],
            mode='text',
            text=year_data['region'],
            textfont=dict(
                size=11,
                color='black',
                family='Arial Black'
            ),
            hoverinfo='skip',
            name=f'{year}_labels',
            visible=(year == years[4])  # 2018년을 기본으로 표시
        ))

    # 슬라이더 생성
    steps = []
    for i, year in enumerate(years):
        step = dict(
            method="update",
            args=[
                {"visible": [False] * len(fig.data)},
                {"title": f"서울시 구별 범죄율 ({year}년)"}
            ],
            label=str(year)
        )
        step["args"][0]["visible"][i*2] = True
        step["args"][0]["visible"][i*2 + 1] = True
        steps.append(step)

    sliders = [dict(
        active=4,  # 2018년 인덱스
        yanchor="top",
        y=0.02,
        xanchor="left",
        x=0.05,
        currentvalue=dict(
            prefix="년도: ",
            visible=True,
            xanchor="left",
            font=dict(size=16, color='#333')
        ),
        pad=dict(b=10, t=10),
        len=0.9,
        steps=steps
    )]

    # 레이아웃 설정
    fig.update_layout(
        mapbox=dict(
            style="open-street-map",
            center=dict(lat=37.5665, lon=126.9780),
            zoom=10
        ),
        sliders=sliders,
        title=dict(
            text=f'서울시 구별 범죄율 ({years[4]}년)',
            font=dict(size=24, color='#333'),
            x=0.5,
            xanchor='center'
        ),
        height=700,
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=False,
        dragmode='zoom'
    )

    print("\n✓ 시각화 생성 완료")
    fig.show()

else:
    print("⚠ 오류: GeoJSON을 로드할 수 없어 지도를 생성할 수 없습니다.")

✓ CSV 로드 완료: 264개 레코드
  연도 범위: 2014 - 2024
  구 개수: 24개
✓ GeoJSON 로드 완료

✓ 시각화 생성 완료
